In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint, uniform

# 1. 加载数据（这里使用示例数据，实际应用时替换为自己的数据）
# 假设我们有一个DataFrame，X是特征，y是目标变量
# 示例数据生成
np.random.seed(42)
X = np.random.rand(1000, 10)  # 1000个样本，10个特征
y = X.dot(np.random.rand(10)) + np.random.normal(0, 0.1, 1000)  # 线性关系加噪声

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 创建基础随机森林回归模型
rf = RandomForestRegressor(random_state=42)

# 3. 设置参数分布用于随机搜索
param_dist = {
    'n_estimators': randint(50, 500),  # 树的数量
    'max_depth': [None] + list(np.arange(5, 30)),  # 树的最大深度
    'min_samples_split': randint(2, 20),  # 分裂内部节点所需的最小样本数
    'min_samples_leaf': randint(1, 20),  # 叶节点所需的最小样本数
    'max_features': ['sqrt', 'log2', None] + list(np.linspace(0.1, 1.0, 10)),  # 寻找最佳分裂时考虑的特征数量
    'bootstrap': [True, False],  # 是否使用bootstrap样本
    'max_samples': [None] + list(np.linspace(0.1, 1.0, 10))  # 如果bootstrap=True，从X抽取的样本比例
}

# 4. 设置随机搜索交叉验证
# n_iter: 参数设置的数量
# cv: 交叉验证折数
# scoring: 评估指标
# n_jobs: 并行运行的作业数(-1表示使用所有处理器)
# verbose: 控制详细程度
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=100,  # 尝试100组不同的参数
    cv=5,  # 5折交叉验证
    scoring='neg_mean_squared_error',  # 评估指标为负均方误差
    random_state=42,
    n_jobs=-1,
    verbose=1
)

# 5. 在训练数据上执行随机搜索
print("开始随机搜索调参...")
random_search.fit(X_train, y_train)

# 6. 输出最佳参数和最佳得分
print("\n最佳参数组合:")
print(random_search.best_params_)
print(f"\n最佳交叉验证分数(负MSE): {random_search.best_score_:.4f}")

# 7. 使用最佳模型进行预测
best_rf = random_search.best_estimator_
y_pred = best_rf.predict(X_test)

# 8. 评估测试集性能
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\n测试集性能:")
print(f"均方误差(MSE): {mse:.4f}")
print(f"R平方(R2): {r2:.4f}")

# 9. 可选：输出特征重要性
feature_importance = best_rf.feature_importances_
print("\n特征重要性:")
for i, importance in enumerate(feature_importance):
    print(f"特征 {i}: {importance:.4f}")

# 10. 可选：可视化部分结果
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], '--r')
plt.xlabel('真实值')
plt.ylabel('预测值')
plt.title('真实值 vs 预测值')
plt.show()